<a href="https://colab.research.google.com/github/Aziz-Benamira/Autonomous-Parking-vehicle/blob/main/lane_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving input_parking1.mp4 to input_parking1.mp4


In [ ]:
import numpy as np
import cv2
from moviepy import editor

def region_selection(image):
    mask = np.zeros_like(image)
    rows, cols = image.shape[:2]
    # 调整 ROI 为停车场区域，通常在图像下半部分
    bottom_left  = [cols * 0.05, rows * 0.9]  # 左下
    top_left     = [cols * 0.05, rows * 0.4]  # 左上
    bottom_right = [cols * 0.3, rows * 0.9]  # 右下
    top_right    = [cols * 0.3, rows * 0.4]  # 右上
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, [vertices], 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def hough_transform(image):
    rho = 1
    theta = np.pi/180
    threshold = 15  # 降低阈值以检测更短的线条
    minLineLength = 10  # 停车位线条可能较短
    maxLineGap = 20  # 停车位线条可能有间隙
    return cv2.HoughLinesP(image, rho=rho, theta=theta, threshold=threshold,
                           minLineLength=minLineLength, maxLineGap=maxLineGap)

def filter_parking_lines(lines):
    parking_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:  # 忽略垂直线
                continue
            slope = (y2 - y1) / (x2 - x1)
            # 停车位线条通常接近水平（坡度接近 0）
            if abs(slope) < 0.2:  # 筛选接近水平的线
                length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                if length > 20:  # 确保线条有一定长度
                    parking_lines.append([x1, y1, x2, y2])
    return parking_lines

def group_parking_spaces(lines, image):
    # 按 y 坐标排序线条（停车位线条通常在相似的 y 高度）
    lines = sorted(lines, key=lambda x: (x[1] + x[3]) / 2)  # 按 y 平均值排序
    parking_spaces = []
    i = 0
    while i < len(lines) - 1:
        line1 = lines[i]
        line2 = lines[i + 1]
        # 计算两条线的 y 坐标差
        y1_avg = (line1[1] + line1[3]) / 2
        y2_avg = (line2[1] + line2[3]) / 2
        # 如果两条线 y 坐标接近，可能是停车位边界
        if abs(y1_avg - y2_avg) < 80:  # 假设停车位宽度在 50 像素内
            parking_spaces.append((line1, line2))
            i += 2
        else:
            i += 1
    return parking_spaces

def draw_parking_spaces(image, parking_spaces, color=[0, 255, 0], thickness=2):
    line_image = np.zeros_like(image)
    for space in parking_spaces:
        line1, line2 = space
        # 画两条线
        cv2.line(line_image, (line1[0], line1[1]), (line1[2], line1[3]), color, thickness)
        cv2.line(line_image, (line2[0], line2[1]), (line2[2], line2[3]), color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

def frame_processor(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grayscale, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    region = region_selection(edges)
    hough_lines = hough_transform(region)
    parking_lines = filter_parking_lines(hough_lines)
    parking_spaces = group_parking_spaces(parking_lines, image)
    result = draw_parking_spaces(image, parking_spaces)
    return result

def process_video(test_video, output_video):
    input_video = editor.VideoFileClip(test_video, audio=False)
    processed = input_video.fl_image(frame_processor)
    processed.write_videofile(output_video, audio=False)

# 示例运行
video_path = 'input_parking.mp4'
process_video(video_path, 'output_parking.mp4')

Moviepy - Building video output_parking.mp4.
Moviepy - Writing video output_parking.mp4



Moviepy - Done !
Moviepy - video ready output_parking.mp4


In [ ]:
import numpy as np
import cv2
from moviepy import editor

def region_selection(image):
    mask = np.zeros_like(image)
    rows, cols = image.shape[:2]
    # 调整ROI为车道前方的梯形区域
    bottom_left = [cols * 0.1, rows * 0.95]
    top_left = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(image, mask)

def hough_transform(image):
    rho = 1
    theta = np.pi/180
    threshold = 30  # 提高阈值减少噪声
    minLineLength = 30  # 检测更长的线段
    maxLineGap = 10
    return cv2.HoughLinesP(image, rho=rho, theta=theta, threshold=threshold,
                           minLineLength=minLineLength, maxLineGap=maxLineGap)

def filter_lane_lines(lines):
    left_lines, right_lines = [], []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 == x1:
            continue  # 忽略垂直线
        slope = (y2 - y1) / (x2 - x1)
        length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
        if abs(slope) < 0.5 or length < 30:  # 过滤接近水平和短线段
            continue
        if slope < 0:
            left_lines.append(line[0])
        else:
            right_lines.append(line[0])
    return left_lines, right_lines

def average_slope_intercept(lines, y_min, y_max):
    if not lines:
        return None
    slopes, intercepts = [], []
    for line in lines:
        x1, y1, x2, y2 = line
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        slopes.append(slope)
        intercepts.append(intercept)
    avg_slope = np.mean(slopes)
    avg_intercept = np.mean(intercepts)
    # 计算线段在y_min和y_max处的x坐标
    x_min = int((y_min - avg_intercept) / avg_slope)
    x_max = int((y_max - avg_intercept) / avg_slope)
    return [x_min, y_min, x_max, y_max]

def draw_lanes(image, left_lane, right_lane):
    line_image = np.zeros_like(image)
    if left_lane is not None:
        x1, y1, x2, y2 = left_lane
        cv2.line(line_image, (x1, y1), (x2, y2), [0, 255, 0], 10)
    if right_lane is not None:
        x1, y1, x2, y2 = right_lane
        cv2.line(line_image, (x1, y1), (x2, y2), [0, 255, 0], 10)
    return cv2.addWeighted(image, 1.0, line_image, 0.8, 0.0)

def color_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # 白色掩膜
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    # 黄色掩膜
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    return cv2.bitwise_or(mask_white, mask_yellow)

def frame_processor(image):
    # 颜色过滤
    mask = color_mask(image)
    color_masked = cv2.bitwise_and(image, image, mask=mask)
    # 灰度化与CLAHE增强
    gray = cv2.cvtColor(color_masked, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    # 边缘检测与ROI
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)
    roi = region_selection(edges)
    # 霍夫变换与车道线处理
    lines = hough_transform(roi)
    if lines is None:
        return image
    left_lines, right_lines = filter_lane_lines(lines)
    y_min = int(image.shape[0] * 0.6)
    y_max = image.shape[0]
    left_lane = average_slope_intercept(left_lines, y_min, y_max)
    right_lane = average_slope_intercept(right_lines, y_min, y_max)
    return draw_lanes(image, left_lane, right_lane)

def process_video(input_video, output_video):
    video = editor.VideoFileClip(input_video, audio=False)
    processed = video.fl_image(frame_processor)
    processed.write_videofile(output_video, audio=False)

# 示例运行
process_video('input_parking.mp4', 'output.mp4')


t:  20%|██        | 314/1543 [20:59<00:41, 29.26it/s, now=None]

t:   5%|▍         | 74/1543 [00:16<01:17, 19.04it/s, now=None]

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4





t:   0%|          | 0/1543 [00:00<?, ?it/s, now=None]

t:   0%|          | 4/1543 [00:00<00:41, 36.79it/s, now=None]

t:   1%|          | 8/1543 [00:00<00:51, 29.86it/s, now=None]

t:   1%|          | 12/1543 [00:00<00:49, 31.01it/s, now=None]

t:   1%|          | 16/1543 [00:00<00:45, 33.78it/s, now=None]

t:   1%|▏         | 20/1543 [00:00<00:44, 34.44it/s, now=None]

t:   2%|▏         | 24/1543 [00:00<00:43, 34.64it/s, now=None]

t:   2%|▏         | 29/1543 [00:00<00:40, 37.67it/s, now=None]

t:   2%|▏         | 35/1543 [00:00<00:35, 41.90it/s, now=None]

t:   3%|▎         | 40/1543 [00:01<00:34, 44.06it/s, now=None]

t:   3%|▎         | 46/1543 [00:01<00:32, 46.59it/s, now=None]

t:   3%|▎         | 51/1543 [00:01<00:33, 44.60it/s, now=None]

t:   4%|▎         | 56/1543 [00:01<00:32, 45.85it/s, now=None]

t:   4%|▍         | 61/1543 [00:01<00:34, 42.42it/s, now=None]

t:   4%|▍         | 66/1543 [00:01<00:39, 37.22it/s, now=None]

t:   5%|▍         | 70/1543 [00:01<00:41, 35.81it

Moviepy - Done !
Moviepy - video ready output.mp4
